In [4]:
import pandas as pd
import numpy as np 
from datetime import datetime

In [6]:
city_id = 1
district_data = pd.read_csv("district.csv")
aq_station_data = pd.read_csv("station.csv")
aq_data = pd.read_csv("airquality.csv")   #READ ONLY ONE COLUMN HERE 
met_data = pd.read_csv("meteorology.csv")

district_ids = np.array(district_data['district_id'])[np.where(np.array(district_data['city_id'])==1)]
district_rows = np.array([i for i in range(len(aq_station_data)) if np.array(aq_station_data['district_id'])[i] in district_ids])
aq_station_ids,aq_dis_ids = np.array(aq_station_data['station_id'])[district_rows],np.array(aq_station_data['district_id'])[district_rows]

In [34]:
met_data.columns

Index(['id', 'time', 'weather', 'temperature', 'pressure', 'humidity',
       'wind_speed', 'wind_direction'],
      dtype='object')

In [42]:
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import CubicSpline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ShuffleSplit

stand= StandardScaler()

In [ ]:
for aqid,disid in zip(aq_station_ids,aq_dis_ids):

    met_rows = np.where(np.isin(met_data['id'],disid))[0]
    met_station= met_data.iloc[met_rows,:]
    met_station = met_station.drop(['pressure'],axis=1)
    
    
    for cols in ['weather','temperature','humidity','wind_speed']:
        csloss = []
        without_NA = met_station.dropna(subset=[cols]).iloc[:,[met_station.columns.get_loc('time'),met_station.columns.get_loc(cols)]].reset_index(drop=True)
        dates = pd.to_datetime([datetime.fromisoformat(without_NA.iloc[i,without_NA.columns.get_loc('time')]) for i in range(len(without_NA))])
        time_differences = ((dates - dates[0]).total_seconds() // 3600)
        indices = ShuffleSplit(n_splits=5, test_size=.2, random_state=0).split(time_differences)
        for i in indices:
            train_indices = list(i[0])
            test_indices = list(i[1])
            xtrain = time_differences[train_indices].sort_values()
  
            
            Fit= stand.fit(np.array(without_NA.iloc[train_indices,without_NA.columns.get_loc(cols)])[xtrain.argsort()].reshape(-1,1))
            ytrain = Fit.transform(np.array(without_NA.iloc[train_indices,without_NA.columns.get_loc(cols)]).reshape(-1,1))
            
            xtest = time_differences[test_indices]
            ytest = Fit.transform(np.array(without_NA.iloc[test_indices,without_NA.columns.get_loc(cols)]).reshape(-1,1))
            
            cs = CubicSpline(xtrain, ytrain)
            y_pred = cs(xtest)
            csloss.append(mean_squared_error(ytest,y_pred))
        
            ## similarly interpolate using other methods.
            
        avg_cs_loss = np.mean(np.array(csloss))
        ## select method with least avg loss for that station, column
            